This script performs data cleaning for various exogenous variables (GDP growth, CPI growth, Unemployment rate, Michigan Sentiment Consumer Index (MSCI)) and builds the dependent variable (log-returns of a equity portfolio).

In [48]:
import pandas as pd
import numpy as np
import os as os
import pickle
import yfinance as yf
import pandas_datareader.data as pdr
from sklearn.preprocessing import StandardScaler, MinMaxScaler


tickers = ['AAPL', 'MSI', 'GOOGL']

price_data = yf.download(tickers, start='2011-11-01', end='2022-12-31', interval='1mo')['Close']

#we have to download the data separately for Samsung as it is not in US prices:
samsung_data = yf.download('005930.KS', start='2011-11-01', end='2022-12-31', interval='1mo')['Close']

#fetching usd/krw historical exchange rate to convert samsung KWR stock price to USD stock price
exchange_rates = pdr.DataReader('DEXKOUS', 'fred', start='2011-11-01', end='2022-12-31')
exchange_rates = exchange_rates.resample('1ME').ffill().reindex(samsung_data.index, method='nearest')
price_data['SAMS'] = samsung_data / exchange_rates['DEXKOUS']
order = ['AAPL', 'SAMS', 'GOOGL', 'MSI']
price_data = price_data[order]



weights = [0.6823, 0.2520, 0.0390, 0.0267]
weighted_prices = price_data.multiply(weights, axis='columns')
portfolio_value = weighted_prices.sum(axis=1)
portfolio_value.to_csv('./tsdata/portfolio.csv', index=True)
log_returns = np.log(portfolio_value / portfolio_value.shift(1)).dropna()

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  1 of 1 completed


In [49]:
class PreprocessMichiganSentiment:
    def __init__(self, data):
        self.data = data

    def preprocess(self):
        self.data["Date"] = pd.to_datetime(self.data["YYYY"].astype(str) + '-' + self.data["Month"], format='%Y-%B')
        self.data = self.data.sort_values("Date")
        self.data = self.data[(self.data["Date"] >= '2011-12-01') & (self.data["Date"] <= '2022-12-31')]
        self.data = self.data.groupby(self.data["Date"].dt.to_period("M")).last()
        self.data.reset_index(drop=True, inplace=True)
        self.data.drop(columns=['Month', 'YYYY'], inplace=True)

        self.data['Date'] = pd.to_datetime(self.data['Date'])
        self.data.set_index('Date', inplace=True)
        return self.data



michigansentiment = pd.read_csv("./tsdata/michigansentiment.csv")
michigansentiment_preprocessed = PreprocessMichiganSentiment(michigansentiment)
michigansentiment = michigansentiment_preprocessed.preprocess()


In [50]:
#Load sentiment scores
sentiment = pd.read_csv("./Sentiment_data/sentiment_scores.csv").drop(["Unnamed: 0"], axis=1)
len(sentiment)

132

In [51]:
#Checking if sentiment scores are complete
pd.set_option('display.max_rows', None)
data = pd.DataFrame()
data["month"] = sentiment["month"]
expected_months = pd.period_range(start='2012-01', end='2022-12', freq='M')
data['month'] = data['month'].astype(str).apply(lambda x: pd.Period(year=int(x[:4]), month=int(x[4:]), freq='M'))
missing_months = expected_months[~expected_months.isin(data['month'])]
missing_months

PeriodIndex([], dtype='period[M]')

In [52]:
#We have to add one additional row to the sentiment data frame to match the index. However, the artificial score will be disregarded later.
new_row = pd.DataFrame({
    "month": [201112],
    "scaled_scores": [sentiment["scaled_scores"].mean()]
})

sentiment = pd.concat([new_row, sentiment], ignore_index=True)
len(sentiment)

133

In [53]:
#Create final data frame
df = pd.DataFrame()

df.index = log_returns.index

df["returns"] = log_returns.values
#df["gdpgrowth"] = gdpgrowth["Monthly Real GDP Index"].values
#df["cpigrowth"] = cpigrowth["USACPALTT01CTGYM"].values
#df["unemp"] = unemploymentrate["UNRATE"].values
df["msci"] = michigansentiment["ICS_ALL"].values
df["sentiment"] = sentiment["scaled_scores"].values


In [54]:
scaler_msci = MinMaxScaler(feature_range = (-1, 1))
df[["msci"]] = scaler_msci.fit_transform(df[["msci"]])
df.to_csv('./tsdata/preprocessed_data.csv', index=True)